In [1]:
import pennylane as qml
from qiskit_ibm_provider import IBMProvider
from mitiq.zne.inference import RichardsonFactory
from mitiq.zne.scaling import fold_global
SIMULATION_MODE = True

##  Build a simple noise model with depolarizing noise

In [2]:
if SIMULATION_MODE:
    noise_strength = 0.05
    dev_noise_free = qml.device("default.mixed", wires=2)
    dev = qml.transforms.insert(
        dev_noise_free,
        qml.DepolarizingChannel,
        noise_strength
    )
elif IBMProvider.saved_accounts() and USE_REAL_HARDWARE:
    provider = IBMProvider()
    dev = qml.device(
        "qiskit.ibmq",
        wires=2,
        backend="ibmq_qasm_simulator",
        provider=provider
    )    

In [3]:
def circuit():
    """
    @A circuit modelling the effect of depolarizing noise in preparing a Bell state
    """
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

Zero-noise extrapolation (ZNE) is a noise mitigation technique. It works by intentionally scaling the noise of a quantum circuit to then extrapolate the zero-noise limit of an observable of interest. In this task, you will build a simple ZNE function from scratch:

- [x] Build a simple noise model with depolarizing noise 
- [ ] Create different circuits to test your noise models and choose the observable to measure 
- [ ] Apply the unitary folding method. 
- [ ] Apply the extrapolation method to get the zero-noise limit. Different extrapolation methods achieve different results, such as Linear, polynomial, and exponential.
- [ ] Compare mitigated and unmitigated results 
- [ ] Bonus: Run your ZNE function in real quantum hardware through the IBM Quantum Service

Check the Mitiq documentation for references. You are not allowed to use the functions from Mitiq or any other frameworks where ZNE is already implemented. 


In [4]:
scale_factors = [1, 2, 3]

device_circuit = qml.QNode(circuit, dev)
error_mitigated_device_circuit = qml.transforms.mitigate_with_zne(
    device_circuit,
    scale_factors,
    folding = fold_global,    
    extrapolate = RichardsonFactory.extrapolate,
)

In [6]:
unmitigated = device_circuit()
mitigated = error_mitigated_device_circuit()
print(f"Unmitigated result {unmitigated}")
print(f"Mitigated result   {mitigated}")

Unmitigated result 0.8711111111111454
Mitigated result   1.1968421399177436


As the ideal, desired result is 1.000, the mitigated result performs much better than unmitigated.

```
for extrapolation_method in [...]:

```